In [8]:
!python -m spacy download en_core_web_sm

[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
import spacy
import re
import pandas as pd
import numpy as np
import requests

import networkx as nx
import matplotlib.pyplot as plt

from common import *

nlp = spacy.load('en_core_web_sm')
pd.set_option('display.max_colwidth', 200)
%matplotlib inline


In [6]:
book = read_file('data/Strang-Linear Algebra.txt')
chapter = dict(get_one_chapter_strang(3, book, subsections=True))


subsection START 26 3.1 Orthogonal Vectors and Subspaces 
subsection END 16707 
Problem Set 3.1 
NEW subsection START 9011 3.2 Cosines and Projections onto Lines 

chapter beginning -  Vectors with xTy = 0
subsection START 9011 3.2 Cosines and Projections onto Lines 

subsection END 20485 
Problem Set 3.2 
NEW subsection START 5058 3.3 Projections and Least Squares 

chapter beginning -  Up to this point, Ax
subsection START 5058 3.3 Projections and Least Squares 

subsection END 27415 
Problem Set 3.3 
NEW subsection START 9468 3.4 Orthogonal Bases and Gram-Schmidt 

chapter beginning -  In an orthogonal bas
subsection START 9468 3.4 Orthogonal Bases and Gram-Schmidt 

subsection END 37514 
Problem Set 3.4 
NEW subsection START 6135 3.5 The Fast Fourier Transform 

chapter beginning -  The Fourier series i
subsection START 6135 3.5 The Fast Fourier Transform 

subsection END 24119 
Problem Set 3.5 


In [7]:
chapter.keys()


dict_keys(['Orthogonal Vectors and Subspaces', 'Cosines and Projections onto Lines ', 'Projections and Least Squares ', 'Orthogonal Bases and Gram-Schmidt ', 'The Fast Fourier Transform '])

In [41]:
### FEATURES

def get_tags(sentence):
    return [token.tag_ for token in nlp(sentence)]

def get_noun_adj_tokens(words):
    return [token.lemma_ for token in nlp(words) 
            if token.pos_ == 'ADJ' or token.pos_ == 'NOUN']

    
# Informative 
def is_first_sentence(f, c):
    return f == c


def has_superlatives(curr):
    pos_tags = get_tags(curr)
    return 'JJR' in pos_tags or 'JJS' in pos_tags


def has_abbreviation(curr):
    is_abbr = lambda word: word.upper() == word and len(word) > 1
    return any(is_abbr(x) for x in curr.split())


# Generative


def number_of_words(curr):
    return len(curr.split())


def common_tokens_count(curr, title):
    curr_tokens = get_noun_adj_tokens(curr)
    title_tokens = get_noun_adj_tokens(title.lower())
    
    return sum([tok in curr_tokens for tok in title_tokens]) \
            / number_of_words(curr))    


def begins_with_discourse_connective(curr):
    discource_connective = ['because', 'since', 'when', 'thus', 
                            'however', 'although', 'for example', 
                            'and', 'for instance']
    curr = curr.lower()
    return any(curr.startswith(x) for x in discource_connective)


def nouns_number(curr):
    return sum(x.tag_ == 'NOUN' for x in curr)


def pronouns_number(curr):
    return sum(x.tag_ == 'PRON' for x in curr)


In [45]:
[x.pos_ for x in nlp('I am a very beautiful boy.')]

['PRON', 'AUX', 'DET', 'ADV', 'ADJ', 'NOUN', 'PUNCT']

In [44]:
begins_with_discourse_connective('it is both an amine and a carboxylic acid')

False

In [24]:
[token.pos_ for token in nlp('The energy levels of electrons')]

['DET', 'NOUN', 'NOUN', 'ADP', 'NOUN']

In [46]:
spacy.explain('PRON')

'pronoun'